# Test/observe merge compatibility between benchmarks

## Setup

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
import config, os, sys

In [2]:
os.listdir(config.INTERM_DIR)

['demo_botiot_clean.csv', 'full_cicids_cleaned.csv', 'full_unsw_cleaned.csv']

In [3]:
bot = pd.read_csv(config.INTERM_DIR / 'demo_botiot_clean.csv') # demo
print('loaded bot')
cicids = pd.read_csv(config.INTERM_DIR / 'full_cicids_cleaned.csv')
print('loaded cicid')
unsw = pd.read_csv(config.INTERM_DIR / 'full_unsw_cleaned.csv')
print('loaded unsw')

C:\Users\taznk\AppData\Local\Temp\ipykernel_16848\2791770745.py:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  bot = pd.read_csv(config.INTERM_DIR / 'demo_botiot_clean.csv') # demo


loaded bot
loaded cicid
loaded unsw


In [4]:
dfs = [bot, cicids, unsw]
[sys.getsizeof(d) // (1_000**2) for d in dfs] # MB

[2780, 1762, 1391]

In [5]:
[len(d) for d in dfs]

[3668522, 2827876, 2059417]

In [6]:
for d, name in zip(dfs, ('bot', 'cicids', 'unsw')):
    print('\n', name)
    print(d.columns)


 bot
Index(['pkSeqID', 'stime', 'flgs', 'flgs_number', 'proto', 'proto_number',
       'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state',
       'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min',
       'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
       'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto',
       'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP',
       'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport',
       'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP',
       'Pkts_P_State_P_Protocol_P_SrcIP', 'attack', 'category', 'subcategory'],
      dtype='object')

 cicids
Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length M

## Merge

UNSW + CICIDS2017

In [7]:
unsw['attack_cat'].unique()

array(['normal', 'exploits', 'reconnaissance', 'dos', 'generic',
       'shellcode', 'fuzzers', 'worms', 'backdoors', 'analysis',
       'backdoor'], dtype=object)

In [8]:
cicids['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [10]:
import pandas as pd

# Rename common columns
unsw_renamed = unsw.rename(columns={
    'dur': 'Duration',
    'sbytes': 'Fwd_Bytes',
    'dbytes': 'Bwd_Bytes',
    'Spkts': 'Fwd_Pkts',
    'Dpkts': 'Bwd_Pkts',
    'smeansz': 'Fwd_Pkt_Len_Mean',
    'dmeansz': 'Bwd_Pkt_Len_Mean',
    'swin': 'Fwd_InitWinBytes',
    'dwin': 'Bwd_InitWinBytes',
    'attack_cat': 'Target'
})

cicids_renamed = cicids.rename(columns={
    'Flow Duration': 'Duration',
    'Total Length of Fwd Packets': 'Fwd_Bytes',
    'Total Length of Bwd Packets': 'Bwd_Bytes',
    'Total Fwd Packets': 'Fwd_Pkts',
    'Total Backward Packets': 'Bwd_Pkts',
    'Fwd Packet Length Mean': 'Fwd_Pkt_Len_Mean',
    'Bwd Packet Length Mean': 'Bwd_Pkt_Len_Mean',
    'Init_Win_bytes_forward': 'Fwd_InitWinBytes',
    'Init_Win_bytes_backward': 'Bwd_InitWinBytes',
    'Label': 'Target'
})

# keep common columns
common_cols = list(set(cicids_renamed.columns).intersection(set(unsw_renamed.columns)))
# common_cols.remove('Label') # now 'Target'

# Add source column
unsw_renamed['source'] = 'UNSW'
cicids_renamed['source'] = 'CICIDS'

# inner merge
# inner_merge = pd.concat([unsw_renamed[common_cols + ['source']], 
                         # cicids_renamed[common_cols + ['source']]], ignore_index=True)

In [11]:
# inner_merge.head()

In [15]:
for d in dfs: del d

# outer merge
outer_merge = pd.concat([unsw_renamed, cicids_renamed], axis=0, ignore_index=True, sort=False)

# readable col order
cols = ['source', 'Target'] + [col for col in outer_merge.columns if col not in ['source', 'Target']]
outer_merge = outer_merge[cols]

outer_merge.head()

# !! use colab

MemoryError: Unable to allocate 37.3 MiB for an array with shape (1, 4887293) and data type object

In [ ]:
outer_merge.to_csv(config.INTERM_DIR / 'outer_merge_demos.csv')